# Order a Basemap Using the Orders API & SDK via AOI & Quad ID

## Overview ##
---
Download Basemap quads based on the mosaic name, merged and clipped to the area of interest.

For example, from Explorer, I've defined my area of interest and downloaded that geoJSON.

I've selected the basemap:

global_monthly_2022_01_mosaic

and downloaded my Area of Interest.

To view the basemap, I clicked on the arrow-in-a-box icon to open that basemap in the Basemap viewer.

To see all quads in my subject area, I uploaded my Area of Interest.

326-1257,327-1257,328-1257,326-1256,327-1256,328-1256,326-1255,327-1255,328-1255


<a name="Get-a-basemap-ID"></a> 
## Get a basemap ID
---

You can get a basemap ID from Planet Explorer or the Basemap Viewer.

### Get a basemap ID from Planet Explorer
You&#8217;ll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open <a href="https://www.planet.com/explorer/#" target="_blank">Planet Explorer</a> and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for basemaps or speak to your customer service manager.)

Once you've selected a basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example monthly or quarterly. Then select one of the resulting basemap IDs, such as "global_monthly_2022_05_mosaic." That's the value you'll be passing into the Orders API. 

### Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the Basemap Viewer and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want.

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as "global_monthly_2022_01_mosaic." That's the value you'll be passing into the Orders API.

<a name="how-to-get-started"></a> 
## Start coding

---
Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [ ]:
import planet
import os

In [ ]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = planet.Auth.from_key(API_KEY)

### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. (change this to talk about tools)

In [ ]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "tools": [
        {"merge": {}},
        {"clip": {}}
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "notebook-practice",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}


### Create a session with SDK and poll for success

Here, we are creating a session with the SDK and creating the order using the order parameters. Then, we are polling for success. This may take a few minutes to complete.



In [ ]:
async def create_and_deliver_order(order_params, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state,
                        max_attempts=0)


In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(order_params, client)

## Use Basemaps API to get the Basemap and quad IDs
---

Use the Basemap APIs to retrieve the basemap you want and the quad IDs. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


In [9]:
import requests
from requests.auth import HTTPBasicAuth
import json

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__contains': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemap_service_response = requests.get(
    url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemap_service_response.raise_for_status()
if basemap_service_response.status_code != 204:
    basemaps = json.loads(basemap_service_response.text)
print(basemaps)


In [ ]:
mosaic_ID = basemaps['mosaics'][0]['id']
mosaic_box = basemaps['mosaics'][0]['bbox']
mosaic_box_str = ','.join(map(str, mosaic_box))

quad_params = {
    'mosaic_id': mosaic_ID,
    'bbox': mosaic_box_str,
}

quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic_ID)
quadServiceResponse = requests.get(
    url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)


### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. 

In [ ]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "point_reyes_rolling_normalized_2020-04-30_mosaic",
            "quad_ids": quadIds
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": 3.0 / 111000.0,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "arctan(b1)",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "notebook-practice",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}


### Create a session with SDK and poll for success

Here, we are creating a session with the SDK and creating the order using the order parameters. Then, we are polling for success. This may take a few minutes to complete.



In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(order_params, client)

### Create a session with SDK and poll for success

After a few simple steps, we have basemaps ordered to our Google Cloud Platform bucket using the SDK v2 Beta. For more information, check out the SDK v2 Beta docs [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 
